# Part 1: Data Processing

### Task 1: Importing and cleaning the a sample of the FakeNewsCorpus

We will be importing the data with pandas cvs method. Cleaning the data will be done through a combination of the clean-text module and manual data cleaning, using regular expressions. 

In [ ]:
import nltk
import pandas as pd
import re
import requests

def clean_text(text: str):
    """cleans raw data using re.sub() to remove double newlines, space, and tabs. Also replace dates, emails, urls, and numbers"""
    text = text.lower()
    
    num_pattern = re.compile(r"(\d+)")
    date_pattern = re.compile(r"((\d{2})-(\d{2})-(\d{4}))") #usind the dd-mm-yyyy format
    email_pattern = re.compile(r"(([\w\-_.]*)(@\w+)(.com))")
    url_pattern = re.compile(r"((https:\/\/www\.)([a-zA-Z0-9]*)(\.com))")
    
    text = re.sub(" +", " ", text)
    text = re.sub("\t+", "\t", text)
    text = re.sub("\n+", "\n", text)
    text = re.sub(date_pattern, "<DATE>", text)
    text = re.sub(email_pattern, "<EMAIL>", text)
    text = re.sub(url_pattern, "<URL>", text)
    text = re.sub(num_pattern, "<NUM>", text)
    
    return text

def word_analasis_text(text: list):
    """returns reduction rate of vocabulary of tokenzied data before and after removing stopwords and stemming"""
    return text


We import the data and run our functions. (Remember to run the functions first)

In [ ]:
#importing the CVS file as raw text
raw_data_fake_news = requests.get("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv").text

#cleaning raw text
cleaned_data_fake_news = clean_text(raw_data_fake_news)

#tokenizing raw text
tokenized_data_fake_news = nltk.word_tokenize(cleaned_data_fake_news)

#doing a word analasis
word_analasis = word_analasis_text(tokenized_data_fake_news)